In [ ]:
# Stock Market Prediction by Recurrent Neural Network on LSTM Model
# https://medium.com/@aniruddha.choudhury94/stock-market-prediction-by-recurrent-neural-network-on-lstm-model-56de700bff68
# Stock Market Prediction by Recurrent Neural Network on LSTM Model.PDF
# /users/akuppam/documents/Hprog/Py/LSTM/"

In [1]:
from pandas import DataFrame
from pandas import Series
from pandas import concat
from pandas import read_csv
from pandas import datetime
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from math import sqrt
from matplotlib import pyplot
import numpy as np
import pandas as pd

Using TensorFlow backend.


In [2]:
dataset = pd.read_csv('expe_train.csv',index_col="date",parse_dates=True)
dataset = dataset.drop(columns = ['Unnamed: 0', 'adjusted'])
dataset.head()

,open,high,low,close,volume
date,,,,,
2007-01-03,41.919998,42.520000,41.320000,42.099998,1703300.0
2007-01-04,42.000000,42.680000,41.939999,42.540001,1285600.0
2007-01-05,42.599998,42.700001,42.160000,42.520000,1389900.0
2007-01-08,42.380001,42.840000,42.240002,42.740002,1523300.0
2007-01-09,42.759998,42.919998,42.500000,42.840000,2661900.0


In [3]:
dataset.shape

(2769, 5)

In [4]:
#Data cleaning
dataset.isna().any()

# Feature Scaling Normalization
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(dataset)

# Creating a data structure with 60 timesteps and 1 output
X_train = []
y_train = []

for i in range(60, len(dataset)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)

# Reshaping
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [5]:
training_set_scaled.shape

(2769, 5)

In [6]:
X_train.shape, y_train.shape

((2709, 60, 1), (2709,))

In [8]:
X_train

array([[[0.20087157],
        [0.20141632],
        [0.20550183],
        ...,
        [0.23124065],
        [0.22756367],
        [0.22293341]],

       [[0.20141632],
        [0.20550183],
        [0.20400382],
        ...,
        [0.22756367],
        [0.22293341],
        [0.22865314]],

       [[0.20550183],
        [0.20400382],
        [0.2065913 ],
        ...,
        [0.22293341],
        [0.22865314],
        [0.23055971]],

       ...,

       [[0.89595534],
        [0.91978756],
        [0.91502114],
        ...,
        [0.73382815],
        [0.73253439],
        [0.73423669]],

       [[0.91978756],
        [0.91502114],
        [0.92094511],
        ...,
        [0.73253439],
        [0.73423669],
        [0.72831266]],

       [[0.91502114],
        [0.92094511],
        [0.92087706],
        ...,
        [0.73423669],
        [0.72831266],
        [0.73396432]]])

In [9]:
y_train

array([0.22865314, 0.23055971, 0.22865314, ..., 0.72831266, 0.73396432,
       0.73859458])

In [ ]:
# FOLLOWING SNIPPET ONLY GOOD FOR 1D/2D ARRAYS, AND NOY 3D ARRAYS

# Series to DF, and then to csv
#X_train = pd.DataFrame(X_train)
#X_train.to_csv("X_train.csv")

# Numpy to csv
#a = np.asarray(X_train)
#np.savetxt("X_train.csv", a, delimiter=",")

# array to DF to csv
# pd.DataFrame(X_train).to_csv("X_train.csv")

In [10]:
import numpy as np
import pandas as pd

# create an example array
a = np.arange(24).reshape([2,3,4])

# convert it to stacked format using Pandas
stacked = pd.Panel(a.swapaxes(1,2)).to_frame().stack().reset_index()
stacked.columns = ['x', 'y', 'z', 'value']

# save to disk
stacked.to_csv('stacked.csv', index=False)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3267: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [47]:
# Convert 3D arrays (X_train) to 'stacked' format using Pandas, and then to csv
stacked = pd.Panel(X_train.swapaxes(1,2)).to_frame().stack().reset_index()
stacked.columns = ['x', 'y', 'z', 'value']

# save to disk
stacked.to_csv('X_train.csv', index=False)

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2963: FutureWarning: 
Panel is deprecated and will be removed in a future version.
The recommended way to represent these types of 3-dimensional data are with a MultiIndex on a DataFrame, via the Panel.to_frame() method
Alternatively, you can use the xarray package http://xarray.pydata.org/en/stable/.
Pandas provides a `.to_xarray()` method to help automate this conversion.

  exec(code_obj, self.user_global_ns, self.user_ns)


In [48]:
# array to DF to csv
pd.DataFrame(y_train).to_csv("y_train.csv")

In [49]:
# Building the RNN LSTM model
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
#Using TensorFlow backend.

In [50]:
# Initialising the RNN
regressor = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
regressor.add(Dropout(0.2))

# Adding a second LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50, return_sequences = True))
regressor.add(Dropout(0.2))

# Adding a fourth LSTM layer and some Dropout regularisation
regressor.add(LSTM(units = 50))
regressor.add(Dropout(0.2))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, epochs = 100, batch_size = 32)

Epoch 1/100
2709/2709 [==============================] - 19s 7ms/step - loss: 0.0164
Epoch 2/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0034
Epoch 3/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0032
Epoch 4/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0029
Epoch 5/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0029
Epoch 6/100
2709/2709 [==============================] - 15s 5ms/step - loss: 0.0025
Epoch 7/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0025
Epoch 8/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0023
Epoch 9/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0021
Epoch 10/100
2709/2709 [==============================] - 15s 5ms/step - loss: 0.0021
Epoch 11/100
2709/2709 [==============================] - 15s 6ms/step - loss: 0.0021
Epoch 12/100
2709/2709 [==============================] - 15s 6

Epoch 94/100
2709/2709 [==============================] - 15s 5ms/step - loss: 6.5871e-04
Epoch 95/100
2709/2709 [==============================] - 15s 5ms/step - loss: 6.6918e-04
Epoch 96/100
2709/2709 [==============================] - 15s 5ms/step - loss: 7.0876e-04
Epoch 97/100
2709/2709 [==============================] - 15s 5ms/step - loss: 6.6272e-04
Epoch 98/100
2709/2709 [==============================] - 15s 5ms/step - loss: 7.9049e-04
Epoch 99/100
2709/2709 [==============================] - 15s 5ms/step - loss: 7.8166e-04
Epoch 100/100
2709/2709 [==============================] - 15s 5ms/step - loss: 6.6010e-04


In [54]:
# Getting the real stock price of 2018
dataset_test = pd.read_csv('expe_test.csv',index_col="date",parse_dates=True)
dataset_test = dataset_test.drop(columns = ['Unnamed: 0', 'adjusted'])
dataset.head()

,open,high,low,close,volume
date,,,,,
2007-01-03,41.919998,42.520000,41.320000,42.099998,1703300.0
2007-01-04,42.000000,42.680000,41.939999,42.540001,1285600.0
2007-01-05,42.599998,42.700001,42.160000,42.520000,1389900.0
2007-01-08,42.380001,42.840000,42.240002,42.740002,1523300.0
2007-01-09,42.759998,42.919998,42.500000,42.840000,2661900.0


In [58]:
# Getting the predicted stock price
dataset_total = pd.concat((dataset['open'], dataset_test['open']), axis = 0)
dataset_total.head()

date
2007-01-03    41.919998
2007-01-04    42.000000
2007-01-05    42.599998
2007-01-08    42.380001
2007-01-09    42.759998
Name: open, dtype: float64

In [62]:
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs.shape

(320,)

In [ ]:
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

In [55]:
# Getting the real stock price of 2018
dataset_test = pd.read_csv('expe_test.csv',index_col="date",parse_dates=True)
dataset_test = dataset_test.drop(columns = ['Unnamed: 0', 'adjusted'])
dataset.head()

# Getting the predicted stock price
dataset_total = pd.concat((dataset['open'], dataset_test['open']), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)

X_test = []

for i in range(60, 80):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

predicted_stock_price = regressor.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

ValueError: non-broadcastable output operand with shape (320,1) doesn't match the broadcast shape (320,5)

In [ ]:
# Visualising the results
plt.plot(real_stock_price, color = 'red', label = 'Actual EXPE Stock Price')
plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted EXPE Stock Price')
plt.title('EXPE Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('EXPE Stock Price')
plt.legend()
plt.show()